# Data Wrangling and Cleaning
by Wilson Lee
Data Set : China Mobile User Gemographics
Source Link : https://www.kaggle.com/chinapage/china-mobile-user-gemographics

In [1]:
# import libraries
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# convert sientific notation to decimals
pd.set_option("display.float_format", lambda x:"%.2f" % x)


### Load & Merge the Data

#### Load single file database

In [2]:
df_app_label = pd.read_csv('../../Data/Raw/China Mobile User Gemographics/app_labels.csv')
df_gender_age_test = pd.read_csv('../../Data/Raw/China Mobile User Gemographics/gender_age_test.csv')
df_gender_age_train = pd.read_csv('../../Data/Raw/China Mobile User Gemographics/gender_age_train.csv')
df_label_categories = pd.read_csv('../../Data/Raw/China Mobile User Gemographics/label_categories.csv')
df_phone_brand_device_model = pd.read_csv('../../Data/Raw/China Mobile User Gemographics/phone_brand_device_model.csv')



#### Load multi-file database

##### Load Event List

In [3]:
list_df_events = [];
list_df_events.append(pd.read_csv("../../Data/Raw/China Mobile User Gemographics/events_split/events_00000", index_col=None, header=None))
for wRoot, wDirs, wFiles in os.walk("../../Data/Raw/China Mobile User Gemographics/events_split"):
    for wFilename in wFiles:
        if "events_00000" != wFilename:
            wCurrentFilename = os.path.join(wRoot, wFilename)
            list_df_events.append(pd.read_csv(wCurrentFilename, index_col=None, header=None))
            
df_events =  pd.concat(list_df_events, axis = 0, ignore_index = True)
df_events.columns = df_events.iloc[0]
df_events = df_events.drop(df_events.index[0])

# de-reference loaded dataframe list
list_df_events = []



c:\users\wilson lee\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,1,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##### Load Application Events

In [4]:
list_df_app_events = [];
list_df_app_events.append(pd.read_csv("../../Data/Raw/China Mobile User Gemographics/app_events_split/app_events_00000", index_col=None, header=None))
for wRoot, wDirs, wFiles in os.walk("../../Data/Raw/China Mobile User Gemographics/app_events_split"):
    for wFilename in wFiles:
        if "app_events_00000" != wFilename:
            wCurrentFilename = os.path.join(wRoot, wFilename)
            list_df_app_events.append(pd.read_csv(wCurrentFilename, index_col=None, header=None))
            
df_app_events =  pd.concat(list_df_app_events, axis = 0, ignore_index = True)
df_app_events.columns = df_app_events.iloc[0]
df_app_events = df_app_events.drop(df_app_events.index[0])

# de-reference loaded dataframe list
list_df_app_events = []

c:\users\wilson lee\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Preview Data

##### Gender & Age Data

In [9]:
df_gender_age_test.head(3)

,device_id
0,1002079943728939269
1,-1547860181818787117
2,7374582448058474277


In [10]:
df_gender_age_train.head(3)

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38


##### App Label Data

In [8]:
df_app_label.head(3)

,app_id,label_id
0,7324884708820027918,251
1,-4494216993218550286,251
2,6058196446775239644,406


##### App Category Labels

In [11]:
df_label_categories.head(3)

,label_id,category
0,1,NaN
1,2,game-game type
2,3,game-Game themes


##### Device Brand and Model

In [13]:
df_phone_brand_device_model.head(3)

,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4


##### Event Data

In [14]:
df_events.head(10)

,event_id,device_id,timestamp,longitude,latitude
1,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
2,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
3,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
4,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
5,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66
6,6,1476664663289716375,2016-05-01 00:27:21,0.00,0.00
7,7,5990807147117726237,2016-05-01 00:15:13,113.73,23.00
8,8,1782450055857303792,2016-05-01 00:15:35,113.94,34.70
9,9,-2073340001552902943,2016-05-01 00:15:33,0.00,0.00
10,10,-8195816569128397698,2016-05-01 00:41:31,119.34,26.04


##### Application Event Data

In [15]:
df_app_events.head(10)

,event_id,app_id,is_installed,is_active
1,2,5927333115845830913,1,1
2,2,-5720078949152207372,1,0
3,2,-1633887856876571208,1,0
4,2,-653184325010919369,1,1
5,2,8693964245073640147,1,1
6,2,4775896950989639373,1,1
7,2,-8022267440849930066,1,0
8,2,9112463267739110219,1,0
9,2,-3725672010020973973,1,0
10,2,7167114343576723123,1,1
